### In processing `Adversarial debiasing`

Ce fichier est dédié à la correction de biais in-processing de notre modèle.
La méthode utilisée est `Adversarial debiasing`.

In [1]:
%load_ext autoreload
%autoreload 2

from dataset_prepare import *
from utils import custom_RFC, print_metrics
import pickle

from matplotlib import pyplot as plt

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets.standard_dataset import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.metrics import ClassificationMetric
from sklearn.metrics import classification_report
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

from IPython.display import Markdown, display

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from tqdm import tqdm
import imblearn

2024-04-09 12:07:22.603366: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 12:07:22.849438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 12:07:22.849496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 12:07:22.886928: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 12:07:22.972305: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = load_dataset()
df

,Num_Acc,trajet,catr,circ,nbv,prof,plan,surf,infra,situ,...,atm,col,catv,obs,obsm,choc,mortal,pieton,sexe_conducteur,age
0,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,14
1,202200000001,5,4,2,2,1,1,1,0,1,...,1,3,2,0,2,1,0,0,1,74
2,202200000002,0,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,34
3,202200000002,4,4,2,2,1,1,1,0,1,...,1,3,3,0,2,8,0,0,1,52
4,202200000003,0,3,-1,2,1,1,1,5,1,...,1,2,3,0,2,1,0,0,1,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126656,202200055300,5,3,2,2,1,2,7,0,3,...,9,6,3,1,0,1,1,0,1,27
126657,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,20
126659,202200055301,5,3,2,2,1,1,1,0,1,...,1,3,3,0,0,8,0,0,0,69
126660,202200055302,1,3,3,4,1,1,1,0,1,...,1,2,5,0,2,1,0,0,1,30


In [3]:
label = 'mortal'

# Récupération des ensembles de train/test
X_train, X_test, y_train, y_test = test_train_sets(df)
df = df.drop(columns='Num_Acc')

In [4]:
dataset_orig_train, dataset_orig_test = prepare_standard_dataset(X_train, y_train, X_test, y_test, label)

In [5]:
privileged_groups=[{'sexe_conducteur': 1}]
unprivileged_groups = [{'sexe_conducteur' : 0}]

In [6]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [7]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)
plain_model.fit(dataset_orig_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 0; iter: 0; batch classifier loss: 3.902865


2024-04-09 12:07:39.953993: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


epoch 0; iter: 200; batch classifier loss: 2.167028
epoch 0; iter: 400; batch classifier loss: 2.065169
epoch 0; iter: 600; batch classifier loss: 1.244217
epoch 0; iter: 800; batch classifier loss: 0.570744
epoch 1; iter: 0; batch classifier loss: 0.654130
epoch 1; iter: 200; batch classifier loss: 0.594497
epoch 1; iter: 400; batch classifier loss: 0.609645
epoch 1; iter: 600; batch classifier loss: 0.603588
epoch 1; iter: 800; batch classifier loss: 0.425337
epoch 2; iter: 0; batch classifier loss: 0.441410
epoch 2; iter: 200; batch classifier loss: 0.551197
epoch 2; iter: 400; batch classifier loss: 0.398407
epoch 2; iter: 600; batch classifier loss: 0.426612
epoch 2; iter: 800; batch classifier loss: 0.478975
epoch 3; iter: 0; batch classifier loss: 0.440024
epoch 3; iter: 200; batch classifier loss: 0.437620
epoch 3; iter: 400; batch classifier loss: 0.377839
epoch 3; iter: 600; batch classifier loss: 0.345303
epoch 3; iter: 800; batch classifier loss: 0.474725
epoch 4; iter: 0; 

In [8]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [9]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.085004
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.014355


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.823874
Test set: Balanced classification accuracy = 0.633825
Test set: Disparate impact = 1.087832
Test set: Equal opportunity difference = 0.080551
Test set: Average odds difference = 0.049111
Test set: Theil_index = 0.073639


In [10]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 4.954613; batch adversarial loss: 0.676279
epoch 0; iter: 200; batch classifier loss: 1.373940; batch adversarial loss: 0.646117
epoch 0; iter: 400; batch classifier loss: 0.901361; batch adversarial loss: 0.593258
epoch 0; iter: 600; batch classifier loss: 0.723112; batch adversarial loss: 0.608463
epoch 0; iter: 800; batch classifier loss: 0.578914; batch adversarial loss: 0.557362
epoch 1; iter: 0; batch classifier loss: 0.621347; batch adversarial loss: 0.617289
epoch 1; iter: 200; batch classifier loss: 0.563149; batch adversarial loss: 0.534972
epoch 1; iter: 400; batch classifier loss: 0.545843; batch adversarial loss: 0.614904
epoch 1; iter: 600; batch classifier loss: 0.527005; batch adversarial loss: 0.607454
epoch 1; iter: 800; batch classifier loss: 0.492137; batch adversarial loss: 0.559514
epoch 2; iter: 0; batch classifier loss: 0.458989; batch adversarial loss: 0.629538
epoch 2; iter: 200; batch classifier loss: 0.579382; batch a

In [11]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [15]:
print_conf_matrix(dataset_orig_test.convert_to_dataframe()[0]['mortal'], dataset_debiasing_test.convert_to_dataframe()[0]['mortal'])

In [16]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.085004
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.014355


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.074816
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.002310


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.823874
Test set: Balanced classification accuracy = 0.633825
Test set: Disparate impact = 1.087832
Test set: Equal opportunity difference = 0.080551
Test set: Average odds difference = 0.049111
Test set: Theil_index = 0.073639


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.804604
Test set: Balanced classification accuracy = 0.638352
Test set: Disparate impact = 1.012199
Test set: Equal opportunity difference = 0.046064
Test set: Average odds difference = 0.026449
Test set: Theil_index = 0.075456
